<div style="text-align: right">Paul Novaes<br>August 2018</div> 

# RSA with e = 3

The goal of this notebook is to present RSA in the special (and common) case where the public exponent is $e=3$.

This simplifies quite a bit the implementation and some of the math. In addition, we will see that it stresses the fact that __RSA is based on the difficulty of computing the $k$-th root of a number modulo $n$__. This difficulty is a bit surprising because the problem is easy over regular integers.

## Public-Key Cryptography

A cryptosystem allows 2 people to communicate in a way that their conversation remains private even in the presence of an eavesdropper.

In a public-key cryptosystem, anybody, for example Alice, who wants to receive private messages produces 2 keys:
* a public key $pk$ that she publishes for anybody to see (and use)
* a secret key $sk$ that she keeps to herself

Anybody, for example Bob, who wants to send a message $m$ to Alice uses an encryption function $E$, and the public-key $pk$ to encrypt the message:

$$c=E(m, pk)$$

$E$ is such that from $c$, it is very difficult to get $m$: $E$ is a __one-way function__, easy to compute but difficult to invert.

Alice uses a decryption function $D$ and her private key $sk$, to decrypt c:

$$m = D(c, sk)$$


## RSA Cryptosystem (with e = 3)

In RSA, with $e = 3$, Alice chooses a number $n=pq$, product of 2 distinct primes of the form $6k + 5$.
* the public key is $pk=n$
* the secret key is $sk=(p, q)$

The encrypting and decrypting functions are defined by:
* $E_n(m) = m^3 \bmod n$
* $D_n(c) = \sqrt[3] c \bmod n$ 

In fact we will show that $\sqrt[3] c \bmod n = c^{(2\phi(n) + 1)/{3}} \bmod n$, where $\phi(n) = (p-1)(q-1)$

This cryptosystem relies on the fact that it is easy for Alice, who knows $(p,q)$, to compute the cubic root of the cypher, but not for Bob.

## Proof

Let's show that $$D_n(E_n(m)) \equiv_n m$$

$D_n(E_n(m)) \equiv_n m^{2\phi(n) + 1}$, and therefore the result is obviously true if $m \equiv_n 0$.

So let's assume that $m \neq 0$, mod $n$. Using the Chinese Remaining Theorem (CRT) and then Fermat's Little Theorem ($m^{\phi(n)} \equiv_p 1$), we have:

$$D_n(E_n(m)) \equiv_p m^{2\phi(n) + 1} \equiv_p m.m^{\phi(n)}.m^{\phi(n)} \equiv_p m$$

Therefore for any $m$, $D_n(E_n(m)) \equiv_p m$ and similarly $D_n(E_n(m)) \equiv_q m$.

Using CRT again, $$D_n(E_n(m)) \equiv_n m$$

## Implementation

In [1]:
def encrypt(m, n):
    assert(m >= 0 and m < n)
    return pow(m, 3, n)

def decrypt(c, p, q):
    n = p * q
    assert(c >= 0 and c < n)
    phi = (p - 1)*(q - 1)
    d = (2 * phi + 1) // 3
    return pow(c, d, n)

## Key Generation 

To test whether a number is (probably) prime we will use Miller-Rabin primality test:

In [2]:
from random import *

# For consistency.
seed(0)

def is_miller_rabin_witness(a, n):
    if pow(a, n - 1, n) != 1:
        return True
    k = n - 1
    while k % 2 == 0:
        k //= 2
        res = pow(a, k, n)
        if res == -1 + n:
            return False
        if res != 1:
            return True
    return False

def is_probable_prime(n):
    for i in range(50):
        a = randint(1, n - 1)
        if is_miller_rabin_witness(a, n):
            return False
    return True

We can generate random moduli using these functions:

In [3]:
def random_6n_5_prime(lo, hi):
    while True:
        n = randint(lo, hi)
        n = 6 * (n // 6) + 5
        if (n < lo):
            continue
        if is_probable_prime(n):
            return n

def generate_random_mod(lo, hi):
    while True:
        p = random_6n_5_prime(lo, hi)
        q = random_6n_5_prime(lo, hi)
        if p != q:
            break
    return p, q

__Example__

In [4]:
p, q = generate_random_mod(10**99, 10**100)
n = p * q
m = randint(0, n)
c = encrypt(m, n)
d = decrypt(c, p, q)
print('m =', m, '\n')
print('c =', c, '\n')
print('d =', d, '\n')

m = 75876111289040343342472704386822816492318228110698493761249436040839133866930987078340493312131554252366677133697928942058986049376644172246769247112460862417435349377731303021695522152768738299326443 

c = 37236657159562720508423609525175784772898695858622668783860604927766049520286119180707383265190876962107404504674678883223520359007116076238695995899980974487126046434478569493795665530764372511127308 

d = 75876111289040343342472704386822816492318228110698493761249436040839133866930987078340493312131554252366677133697928942058986049376644172246769247112460862417435349377731303021695522152768738299326443 



## p, q = 5, 11

Let's look at all the cyphers for the smallest possible modulus. We note that:
* we always get the original message after decrypting the cypher
* cyphers seem randomly distributed, and most messages look well scrambled

On the negative side, several messages (9 out of 55) are "fixed points", that is the cypher is identical to the original message!

In [5]:
p, q = 5, 11
n = p * q
print('        message          cypher         decoded')
print()
for m in range(n):
    c = encrypt(m, n)
    d = decrypt(c, p, q)
    assert(m == d)
    if c == d:
        print("%15d %15d %15d =" % (m, c, d))   
    else:
        print("%15d %15d %15d " % (m, c, d))   

        message          cypher         decoded

              0               0               0 =
              1               1               1 =
              2               8               2 
              3              27               3 
              4               9               4 
              5              15               5 
              6              51               6 
              7              13               7 
              8              17               8 
              9              14               9 
             10              10              10 =
             11              11              11 =
             12              23              12 
             13              52              13 
             14              49              14 
             15              20              15 
             16              26              16 
             17              18              17 
             18               2              18 
             19 

## Fixed Points

If we explore this issue further, it seems to dissapear with bigger moduli. For $p, q = 1013, 1019$, we get 9 fixed points again, but out of more than 1 million possible messages.

In [6]:
def print_fixed_points(p, q):
    n = p * q
    print('p =', p, 'q =', q)
    print()
    print('        message          cypher         decoded')
    print()
    for m in range(n):
        c = encrypt(m, n)
        d = decrypt(c, p, q)
        assert(m == d)
        if c == d:
            print("%15d %15d %15d =" % (m, c, d))       

p, q = 101, 107
print_fixed_points(p, q)

p = 101 q = 107

        message          cypher         decoded

              0               0               0 =
              1               1               1 =
           1818            1818            1818 =
           1819            1819            1819 =
           3637            3637            3637 =
           7170            7170            7170 =
           8988            8988            8988 =
           8989            8989            8989 =
          10806           10806           10806 =


In [7]:
p, q = 1013, 1019
print_fixed_points(p, q)

p = 1013 q = 1019

        message          cypher         decoded

              0               0               0 =
              1               1               1 =
         172210          172210          172210 =
         172211          172211          172211 =
         344421          344421          344421 =
         687826          687826          687826 =
         860036          860036          860036 =
         860037          860037          860037 =
        1032246         1032246         1032246 =


## Notes

__Padding__

For short messages $m$ such that $m^3 < n$, $c= m^3$ is easy to invert, even without knowing $p$ and $q$. That's why messages should be padded.

Padding by a random number is also a good general technique so that if the same message gets sent twice, the random padding turns them into 2 different cyphers.

__General $e$__

We could choose $e$ different from $3$. In this case, we need 2 conditions:
* $n$ is square-free
* $(e, \phi(n)) = 1$

If these conditions are met, a decrypting exponent $d$ can be computed from $e$ and $\phi(n)$, using the Euclidean extended algorithm. This exponent is typically computed only once and becomes the secret key.